In [52]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score,StratifiedKFold,RepeatedStratifiedKFold, cross_validate
from copy import deepcopy

from time import time
from sklearn.preprocessing import FunctionTransformer

In [60]:
def train_model(list_model,X_train,y_train,X_test,y_test,metric,cv,scorer,pipeline):
    df_model = pd.DataFrame(columns = ["model_name","set_data","score","model"])
    set_data = ["test","cv","train"]

    for m in list_model: 
        pipeline_copy = deepcopy(pipeline)
        pipeline_copy.set_params(model = list_model[m])
        spot_check = cross_val_score(pipeline_copy,X_train,y_train,cv = cv,scoring = scorer,n_jobs= -1)
        spot_check_acc = cross_val_score(pipeline_copy,X_train,y_train,cv = cv,scoring = "accuracy",n_jobs= -1)
        spot_check = spot_check.mean()
        spot_check_acc = spot_check_acc.mean()
        model = pipeline_copy.fit(X_train,y_train)
        score = metric(y_test,model.predict(X_test),average = "macro")
        score_train = metric(y_train,model.predict(X_train),average = "macro")
        acc_score = model.score(X_test,y_test)
        model_list = [m] * 3
        tes = pd.DataFrame(list(zip(model_list,set_data,[score,spot_check,score_train],[model,model,model])),columns = ["model_name","set_data","score","model"])
        print(f"model {m} selesai di training")
        print(f"score test {score}")
        print(f"score cv {spot_check}")
        print(f"score train {score_train}")
        print(f"acc score test {acc_score}")
        print(f"acc score cv {spot_check_acc}")
        print("=====================================")
        df_model = pd.concat([df_model,tes],ignore_index = True)
        
    return df_model

def rfecv(X, y, pipeline,min_features_to_select=3, cv = 3,step=3,scoring_metric="f1",scoring_decimals=3,random_state=42,groups = None):
    # Initialize survivors and ranked list
    estimator = deepcopy(pipeline)
    estimator.steps.pop(-1)
    survivors = estimator.fit_transform(X,y).columns.tolist()
    ranks = []
    scores = []
    while len(survivors) >= min_features_to_select:
        remove_column_transformer = FunctionTransformer(lambda x: x.drop(ranks, axis=1))
        estimator = deepcopy(pipeline)
        estimator.steps.insert(-1, ('remove_column_transformer', remove_column_transformer))
        print("[%.2f] evaluating %i features ..." % (time(), len(survivors)))
        cv_result = cross_validate(estimator, X, y,
                                cv=cv,
                                groups = groups,
                                scoring=scoring_metric,
                                return_estimator=True)
        score = np.mean(cv_result["test_score"])
        if scoring_decimals is None:
            scores.append(score)
        else:
            scores.append(round(score, scoring_decimals))            
        print("[%.2f] ... score %f." % (time(), scores[-1]))
        
        best_estimator = cv_result["estimator"][np.argmax(cv_result["test_score"])]
        if isinstance(best_estimator, Pipeline):
            weights = best_estimator[-1].feature_importances_
        else:
            weights = best_estimator.feature_importances_
        weights = list(np.power(weights, 2))
        for _ in range(max(min(step, len(survivors) - min_features_to_select), 1)):
            idx = np.argmin(weights)
            ranks.insert(0, survivors.pop(idx))
            weights.pop(idx)
    ranks_reverse = list(reversed(ranks))
    last_max_idx = len(scores) - np.argmax(list(reversed(scores))) - 1
    removed_features = set(ranks_reverse[0:last_max_idx * step])
    best_features = [f for f in X.columns if f not in removed_features]
    return best_features, max(scores), ranks, scores

In [61]:
df = pd.read_csv('../../data/raw/DataTrain_Preliminary.csv',sep = ";",na_values = ['*',99999])
df_test = pd.read_csv('../../data/raw/Data_Prediction.csv',sep = ";")
features = df.columns[0:-1]
df = df.drop_duplicates(subset=features, keep=False)
df.shape
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
0,0.0,tcp,private,SH,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.01,0.94,0.95,0.0,0.95,1.0,0.0,0.0,nmap
1,0.0,tcp,private,S0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.02,0.08,0.00,0.0,1.00,1.0,0.0,0.0,neptune
2,0.0,tcp,http,SF,285.0,3623.0,0.0,0.0,0.0,0.0,...,228.0,1.00,0.00,0.01,NaN,0.00,0.0,0.0,0.0,normal
3,0.0,tcp,http,SF,232.0,584.0,0.0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.17,NaN,0.00,0.0,0.0,0.0,normal
4,1.0,tcp,smtp,SF,1080.0,327.0,0.0,0.0,0.0,0.0,...,154.0,0.58,0.02,0.00,NaN,0.00,0.0,0.0,0.0,normal


In [62]:
## stratified shuffle
X = df.drop(columns="type_of_attack")

#label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df["type_of_attack"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((88044, 41), (22012, 41), (88044,), (22012,))

In [68]:
#make custom transformer
from sklearn.base import BaseEstimator, TransformerMixin

def value_impute(data):
    #if value is nan, then impute with 0, if value = inf, replcae with -1
    if data == np.Inf:
        data = -1
    if data == np.nan:
        data = 0
    return data

class FeatureEngineering(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        X_ = X.copy()
        X_['src_bytes_per_duration'] = X_['src_bytes']/X_['duration']
        X_['src_bytes_per_duration'] = X_.src_bytes_per_duration.apply(lambda x : value_impute(x))
        
        X_['dst_bytes_per_duration'] = X_['dst_bytes']/X_['duration']
        X_['dst_bytes_per_duration'] = X_.dst_bytes_per_duration.apply(lambda x:value_impute(x))

        X_['total_bytes_per_duration'] = (X_['src_bytes'] + X_["dst_bytes"]) / X_['duration']
        X_['total_bytes_per_duration'] = X_['total_bytes_per_duration'].apply(lambda x:value_impute(x))

        X_['num_access_use'] = X_['num_root'] + X_['num_file_creations'] + X_['num_shells'] + X_['num_access_files'] + X_['num_outbound_cmds']
        X_['num_access_use'] = X_.num_access_use.apply(lambda x:value_impute(x))

        X_['total_bytes'] = X_['src_bytes'] + X_['dst_bytes']
        X_['connection_rate'] = X_['count'] / X_['duration']
        X_['connection_rate'] = X_['connection_rate'].apply(lambda x:value_impute(x))

        X_['host_error_rate'] = X_["dst_host_serror_rate"] + X_['dst_host_rerror_rate']
        X_['success_login_ratio'] = X_['num_compromised'] / (X_['num_compromised'] + X_['num_failed_logins'])
        X_['success_login_ratio'] = X_['success_login_ratio'].apply(lambda x:value_impute(x))

        X_['connection_stability'] = X_[['serror_rate','rerror_rate','srv_serror_rate','srv_rerror_rate']].std(axis = 1)
        X_['flag_rate']  = X_['count'] * (X_['serror_rate'] + X_['rerror_rate'] + X_['srv_serror_rate'] + X_['srv_rerror_rate'])

        return X_

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer
#one hot encoder
from sklearn.preprocessing import OneHotEncoder,TargetEncoder

cat_columns = X_train.select_dtypes(include=['object']).columns.tolist()
num_columns = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
feat_eng_cols = ['src_bytes_per_duration','dst_bytes_per_duration','total_bytes_per_duration','num_access_use','total_bytes',
                 'connection_rate','host_error_rate','success_login_ratio','connection_stability','flag_rate']
num_columns = num_columns + feat_eng_cols

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", PowerTransformer())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", TargetEncoder(target_type="continuous",random_state=42))
])

preprocessor = ColumnTransformer([
("numerical",num_pipe,num_columns),
("categorical",cat_pipe,cat_columns)
],remainder = "drop",verbose_feature_names_out=False).set_output(transform="pandas")

In [70]:
prep_pipeline = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocessor",preprocessor)
])
prep_pipeline.fit_transform(X_train,y_train)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,num_access_use,total_bytes,connection_rate,host_error_rate,success_login_ratio,connection_stability,flag_rate,protocol_type,service,flag
37991,-0.307969,-1.134941,-0.920574,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,-1.130042,-0.325724,1.243288,-0.075465,1.236558,1.238821,3.207384,2.653904,3.146027
9261,-0.307969,1.263017,-0.920574,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,0.839250,-0.325724,1.147782,-0.075465,-0.806005,-0.848228,3.207384,3.617773,3.914765
27008,-0.307969,0.758474,0.910892,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,1.229286,-0.115092,...,-0.156468,0.640921,-0.325724,-0.903882,-0.075465,-0.806005,-0.848228,3.206033,3.852263,3.911250
49884,3.369467,0.617948,0.614867,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,0.407211,3.112262,-0.903882,-0.075465,-0.806005,-0.848228,3.938496,4.440353,3.914765
85361,-0.307969,-1.134941,-0.920574,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,-1.130042,-0.325724,0.399694,-0.075465,1.236558,1.352898,3.209313,2.576331,2.034315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17284,-0.307969,0.839891,0.754944,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,1.229286,-0.115092,...,-0.156468,0.583167,-0.325724,-0.903882,-0.075465,-0.806005,-0.848228,3.206033,3.852263,3.911250
79722,-0.307969,1.221190,0.876026,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,1.229286,-0.115092,...,-0.156468,0.862692,-0.325724,-0.903882,-0.075465,-0.806005,-0.848228,3.207384,3.939431,3.914765
40873,-0.307969,-0.044170,-0.920574,-0.060203,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,-0.285717,-0.325724,-0.903882,-0.075465,-0.806005,-0.848228,3.644563,1.831253,3.911250
47348,-0.307969,-1.134941,-0.920574,14.897002,-0.116077,-0.057251,-0.160122,-0.075631,-0.814661,-0.115092,...,-0.156468,-1.130042,-0.325724,1.243288,-0.075465,1.236558,1.351598,3.207442,2.660380,2.032085


In [71]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

pipeline = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("model",None)
])

list_model = {
    "lgbm":LGBMClassifier(verbose=-1,random_state=42),
    "xgb":XGBClassifier(verbosity=0,random_state=42,tree_method="gpu_hist",gpu_id=1),
   # "catB" : CatBoostClassifier(verbose=0,n_estimators=200,random_seed=42,devices="gpu"),
}

from sklearn.metrics import f1_score,make_scorer
f1_scorer = make_scorer(f1_score,average = "macro")

cv = StratifiedKFold(n_splits=5,shuffle=False)
df_model = train_model(list_model,X_train,y_train,X_test,y_test,f1_score,cv,f1_scorer,pipeline)

model lgbm selesai di training
score test 0.973120902126283
score cv 0.9751522738805788
score train 0.9900993411339969
acc score test 0.9962747592222424
acc score cv 0.9965244651674212
model xgb selesai di training
score test 0.9727206176237437
score cv 0.975943671093231
score train 0.9862228376060374
acc score test 0.9962747592222424
acc score cv 0.9966380447413293


In [15]:
X_test_cpy = X_test.copy()
X_test_cpy['predict'] = df_model.model[0].predict(X_test)
X_test_cpy['actual'] = y_test
X_test_cpy['predict'] = le.inverse_transform(X_test_cpy['predict'])
X_test_cpy['actual'] = le.inverse_transform(X_test_cpy['actual'])

#show the observation that is wrongly classified
X_test_cpy[X_test_cpy['predict'] != X_test_cpy['actual']].to_csv("../../data/interim/wrongly_classified.csv",index = False)

In [9]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

pipeline = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("model",None)
])

list_model = {
    "lgbm":LGBMClassifier(verbose=-1,random_state=42),
    "xgb":XGBClassifier(verbosity=0,random_state=42,tree_method="gpu_hist",gpu_id=1),
    "catB" : CatBoostClassifier(verbose=0,n_estimators=200,random_seed=42,devices="gpu"),
}

from sklearn.metrics import f1_score,make_scorer
f1_scorer = make_scorer(f1_score,average = "macro")

cv = StratifiedKFold(n_splits=5,shuffle=False)
df_model = train_model(list_model,X_train,y_train,X_test,y_test,f1_score,cv,f1_scorer,pipeline)

model lgbm selesai di training
score test 0.973120902126283
score cv 0.9751522738805788
score train 0.9900993411339969
model xgb selesai di training
score test 0.9727206176237437
score cv 0.975943671093231
score train 0.9862228376060374
model catB selesai di training
score test 0.9724801846295884
score cv 0.9729820231608584
score train 0.9807517926110468


In [ ]:
from lightgbm import LGBMClassifier

pipeline_rfe_lgb = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("model",LGBMClassifier(verbose=-1,random_state=42))
])
cv = StratifiedKFold(n_splits=5,shuffle=False)
rfe_lgb = rfecv(X_train,y_train,pipeline_rfe_lgb,scoring_metric=f1_scorer,scoring_decimals=8,cv=cv,step=2,min_features_to_select=30)

In [47]:
print(rfe_lgb[0])

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_file_creations', 'num_shells', 'is_host_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [46]:
import optuna
from sklearn.metrics import classification_report

def tuning_model(objective,pipeline,n_trial = 100):
    study = optuna.create_study(direction="maximize")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=n_trial,show_progress_bar=True,)
    best_params = study.best_params
    pipeline.set_params(**best_params)
    pipeline.fit(X_train, y_train)
    test_score = f1_score(y_test, pipeline.predict(X_test),average = "macro")
    train_score = f1_score(y_train, pipeline.predict(X_train),average = "macro")
    valid_score = study.best_value
    y_pred = pipeline.predict(X_test)
    print("best params:", best_params)
    print("train score:", train_score)
    print("test score:", test_score)
    print("valid score:", valid_score)
    print(classification_report(y_test, y_pred,digits=5))
    return pipeline

d:\conda\envs\lomba-dac\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [73]:
best_cols_lgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 
                 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 
                 'num_root', 'num_file_creations', 'num_shells', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 
                 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
                 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
                 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

drop_cols_lgb = [col for col in X_train.columns if col not in best_cols_lgb]

pipeline_rfe_lgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_lgb,axis = 1))),
    ("algo",LGBMClassifier(verbose=-1,random_state=42))
])

def objective(trial):
    #n_estimators = trial.suggest_int("algo__n_estimators", 100, 500, step=100)
    col_sample_bytree = trial.suggest_float("algo__colsample_bytree", 0.1, 1.0)
    subsample = trial.suggest_float("algo__subsample", 0.1, 1.0)
    reg_alpha = trial.suggest_float("algo__reg_alpha", 1e-5, 1e-1, log=True)
    reg_lambda = trial.suggest_float("algo__reg_lambda", 1e-5, 1e-1, log=True)
    max_depth = trial.suggest_int("algo__max_depth", 1, 10)
    max_bin = trial.suggest_int("algo__max_bin", 200, 1000)
    
    #learning_rate = trial.suggest_float("algo__learning_rate", 1e-5, 1e-1, log=True)
    model = LGBMClassifier(max_depth=max_depth, 
                          #  n_estimators=n_estimators,
                          # learning_rate=learning_rate, 
                           colsample_bytree=col_sample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           max_bin=max_bin,
                           random_state=42, 
                           n_jobs=-1,
                           verbose = -1,)
    pipeline_rfe_lgb_tune.set_params(algo=model)
    return cross_val_score(pipeline_rfe_lgb_tune, X_train, y_train, cv=cv, scoring=f1_scorer).mean()

pipeline_lgbm = tuning_model(objective,pipeline_rfe_lgb_tune,n_trial = 30)

Best trial: 2. Best value: 0.975984:  13%|█▎        | 4/30 [02:13<14:25, 33.27s/it]


[W 2023-10-01 20:33:47,603] Trial 4 failed with parameters: {'algo__colsample_bytree': 0.16924599954905534, 'algo__subsample': 0.10846644831147675, 'algo__reg_alpha': 1.55101676733483e-05, 'algo__reg_lambda': 0.019600659336465508, 'algo__max_depth': 8, 'algo__max_bin': 421} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\conda\envs\lomba-dac\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\VICTUS\AppData\Local\Temp\ipykernel_12900\2323638350.py", line 39, in objective
    return cross_val_score(pipeline_rfe_lgb_tune, X_train, y_train, cv=cv, scoring=f1_scorer).mean()
  File "d:\conda\envs\lomba-dac\lib\site-packages\sklearn\model_selection\_validation.py", line 562, in cross_val_score
    cv_results = cross_validate(
  File "d:\conda\envs\lomba-dac\lib\site-packages\sklearn\utils\_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "d

KeyboardInterrupt: 

In [ ]:
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,make_scorer
f1_scorer = make_scorer(f1_score,average = "macro")


pipeline_rfe_xgb = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("model",XGBClassifier(verbosity=0,random_state=42,tree_method="gpu_hist"))
])
cv = StratifiedKFold(n_splits=5,shuffle=False)
rfe_xgb = rfecv(X_train,y_train,pipeline_rfe_xgb,scoring_metric=f1_scorer,scoring_decimals=8,cv=cv,step=2,min_features_to_select=30)

In [19]:
print(rfe_xgb[1])
print(rfe_xgb[0])

0.97634782
['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_shells', 'is_host_login', 'count', 'srv_count', 'serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [51]:
## XGBoost
import optuna

best_cols_xgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot',
                  'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_shells', 'is_host_login', 'count', 
                  'srv_count', 'serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
                  'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
                  'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']
best_cols_xgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']
drop_cols_xgb = [col for col in X_train.columns if col not in best_cols_xgb]

pipeline_rfe_xgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ('prep', preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_xgb,axis = 1))),
    ('algo', XGBClassifier(random_state=42, n_jobs=-1))
])

def objective(trial):
   # n_estimators = trial.suggest_int("algo__n_estimators", 100, 1000, step=100)
    max_depth = trial.suggest_int("algo__max_depth", 1, 10)
    reg_alpha = trial.suggest_float("algo__reg_alpha", 1e-5, 1e-1, log=True)
    reg_lambda = trial.suggest_float("algo__reg_lambda", 1e-5, 1e-1, log=True)
    colsample_bytree = trial.suggest_float("algo__colsample_bytree", 0.1, 1.0)
    gamma = trial.suggest_float("algo__gamma", 1e-5, 1e-1, log=True)
    model = XGBClassifier(max_depth=max_depth, 
                          gamma=gamma,
                          reg_alpha=reg_alpha,
                          reg_lambda=reg_lambda,
                          colsample_bytree=colsample_bytree,
                           random_state=42, n_jobs=-1,
                           tree_method = "gpu_hist",
                           )
    pipeline_rfe_xgb_tune.set_params(algo=model)
    return cross_val_score(pipeline_rfe_xgb_tune, X_train, y_train, cv=cv, scoring=f1_scorer).mean()

pipeline_xgb = tuning_model(objective,pipeline_rfe_xgb_tune,30)

  0%|          | 0/30 [00:00<?, ?it/s]

Best trial: 23. Best value: 0.975114: 100%|██████████| 30/30 [11:31<00:00, 23.04s/it]


best params: {'algo__max_depth': 4, 'algo__reg_alpha': 0.00040717338276114926, 'algo__reg_lambda': 0.000975237644841343, 'algo__colsample_bytree': 0.825606327123948, 'algo__gamma': 6.618221857652686e-05}
train score: 0.984265106631578
test score: 0.9737641424824068
valid score: 0.9751138238832082
              precision    recall  f1-score   support

           0    0.99701   0.99107   0.99403       336
           1    0.91653   0.97340   0.94411       564
           2    0.99959   1.00000   0.99979      7304
           3    0.92627   0.79762   0.85714       252
           4    0.99950   0.99967   0.99958     11998
           5    1.00000   0.99809   0.99904       523
           6    0.99820   0.99462   0.99641       558
           7    1.00000   1.00000   1.00000       472

    accuracy                        0.99650     22007
   macro avg    0.97964   0.96931   0.97376     22007
weighted avg    0.99652   0.99650   0.99643     22007



In [ ]:
Best trial: 22. Best value: 0.977029: 100%|██████████| 30/30 [17:26<00:00, 34.90s/it]
best params: {'algo__max_depth': 10, 'algo__reg_alpha': 0.006570999279081406, 'algo__reg_lambda': 0.0015967627181379426, 'algo__colsample_bytree': 0.9012229217837533, 'algo__gamma': 0.00541634111409239}
train score: 0.9915338170347747
test score: 0.9712022221252201
valid score: 0.9770285516898067
              precision    recall  f1-score   support

           0    0.99107   0.98813   0.98960       337
           1    0.91837   0.95745   0.93750       564
           2    0.99973   0.99973   0.99973      7305
           3    0.89778   0.80159   0.84696       252
           4    0.99925   0.99958   0.99942     12001
           5    1.00000   1.00000   1.00000       523
           6    0.99642   0.99642   0.99642       558
           7    1.00000   1.00000   1.00000       472

    accuracy                        0.99605     22012
   macro avg    0.97533   0.96786   0.97120     22012
weighted avg    0.99601   0.99605   0.99599     22012

In [75]:
best_cols_lgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_root', 'num_file_creations', 'num_shells', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

drop_cols_lgb = [col for col in X_train.columns if col not in best_cols_lgb]
best_params_lgb = {'algo__colsample_bytree': 0.7511822907836249, 'algo__subsample': 0.3149295796873115, 'algo__reg_alpha': 0.00035731264003581947, 'algo__reg_lambda': 0.0034764373812413854, 'algo__max_depth': 8, 'algo__max_bin': 377}

pipeline_rfe_lgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_lgb,axis = 1))),
    ("algo",LGBMClassifier(verbose=-1,random_state=42))
])

pipeline_rfe_lgb_tune.set_params(**best_params_lgb)

cross_val_score(pipeline_rfe_lgb_tune, X_train, y_train, cv=cv, scoring="accuracy").mean()

0.9967175508300876

In [77]:
# pipeline_rfe_lgb_tune.fit(X_train,y_train)
print(classification_report(y_test,pipeline_rfe_lgb_tune.predict(X_test),digits=5))

              precision    recall  f1-score   support

           0    0.99405   0.99110   0.99257       337
           1    0.91511   0.95567   0.93495       564
           2    0.99986   0.99973   0.99979      7305
           3    0.89238   0.78968   0.83789       252
           4    0.99900   0.99983   0.99942     12001
           5    0.99809   0.99809   0.99809       523
           6    1.00000   0.99283   0.99640       558
           7    1.00000   1.00000   1.00000       472

    accuracy                        0.99591     22012
   macro avg    0.97481   0.96587   0.96989     22012
weighted avg    0.99587   0.99591   0.99584     22012



In [78]:
best_cols_xgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_shells', 'is_host_login', 'count', 'srv_count', 'serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

drop_cols_xgb = [col for col in X_train.columns if col not in best_cols_xgb]
best_params_xgb = {'algo__max_depth': 10, 'algo__reg_alpha': 0.006570999279081406, 'algo__reg_lambda': 0.0015967627181379426, 'algo__colsample_bytree': 0.9012229217837533, 'algo__gamma': 0.00541634111409239}

pipeline_rfe_xgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ('prep', preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_xgb,axis = 1))),
    ('algo', XGBClassifier(random_state=42, n_jobs=-1,tree_method = "gpu_hist"))
])

pipeline_rfe_xgb_tune.set_params(**best_params_xgb)
cross_val_score(pipeline_rfe_xgb_tune, X_train, y_train, cv=cv, scoring="accuracy").mean()


0.9967629827886585

In [79]:
pipeline_rfe_xgb_tune.fit(X_train,y_train)
print(classification_report(y_test,pipeline_rfe_xgb_tune.predict(X_test),digits=5))

              precision    recall  f1-score   support

           0    0.99107   0.98813   0.98960       337
           1    0.91837   0.95745   0.93750       564
           2    0.99973   0.99973   0.99973      7305
           3    0.89778   0.80159   0.84696       252
           4    0.99925   0.99958   0.99942     12001
           5    1.00000   1.00000   1.00000       523
           6    0.99642   0.99642   0.99642       558
           7    1.00000   1.00000   1.00000       472

    accuracy                        0.99605     22012
   macro avg    0.97533   0.96786   0.97120     22012
weighted avg    0.99601   0.99605   0.99599     22012



In [80]:
from sklearn.ensemble import StackingClassifier,VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

X_train_cat = X_train.select_dtypes(include=['object'])
X_train_num = X_train.select_dtypes(include=['int64','float64'])

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", PowerTransformer())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", TargetEncoder(target_type="continuous"))
])

preprocessor_base = ColumnTransformer([
("numerical",num_pipe,X_train_num.columns),
("categorical",cat_pipe,X_train_cat.columns)
],remainder = "drop",verbose_feature_names_out=False)

pipeline_lgb_base = Pipeline([
    ("preprocess",preprocessor_base),
    ("algo",LGBMClassifier(verbose=-1,random_state=42))
])
pipeline_xgb_base = Pipeline([
     ("preprocess",preprocessor_base),
     ("algo",XGBClassifier(random_state=42, n_jobs=-1,tree_method = "gpu_hist"))
    ])

pipeline_catb_base = Pipeline([
    ("preprocess",preprocessor_base),
    ("algo",CatBoostClassifier(verbose=0,n_estimators=300,random_seed=42,devices="gpu"))
])

best_cols_lgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_root', 'num_file_creations', 'num_shells', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

drop_cols_lgb = [col for col in X_train.columns if col not in best_cols_lgb]
best_params_lgb = {'algo__colsample_bytree': 0.7511822907836249, 'algo__subsample': 0.3149295796873115, 'algo__reg_alpha': 0.00035731264003581947, 'algo__reg_lambda': 0.0034764373812413854, 'algo__max_depth': 8, 'algo__max_bin': 377}

pipeline_rfe_lgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ("preprocess",preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_lgb,axis = 1))),
    ("algo",LGBMClassifier(verbose=-1,random_state=42))
])

pipeline_rfe_lgb_tune.set_params(**best_params_lgb)

## XGBoost
import optuna

best_cols_xgb = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'num_shells', 'is_host_login', 'count', 'srv_count', 'serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

drop_cols_xgb = [col for col in X_train.columns if col not in best_cols_xgb]
best_params_xgb = {'algo__max_depth': 10, 'algo__reg_alpha': 0.006570999279081406, 'algo__reg_lambda': 0.0015967627181379426, 'algo__colsample_bytree': 0.9012229217837533, 'algo__gamma': 0.00541634111409239}

pipeline_rfe_xgb_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ('prep', preprocessor),
    ("drop_cols",FunctionTransformer(lambda x: x.drop(drop_cols_xgb,axis = 1))),
    ('algo', XGBClassifier(random_state=42, n_jobs=-1,tree_method = "gpu_hist"))
])

pipeline_rfe_xgb_tune.set_params(**best_params_xgb)

best_params_catb = {'algo__max_depth': 8, 'algo__colsample_bylevel': 0.6986321466355678}
pipeline_catb_rfe_tune = Pipeline([
    ("feature_engineering",FeatureEngineering()),
    ('prep', preprocessor),
    ("algo",CatBoostClassifier(verbose=0,n_estimators=100,random_seed=42,devices="gpu"))
])

pipeline_catb_rfe_tune.set_params(**best_params_catb)

estimators = [
    ("lgbm",pipeline_lgb_base),
    ("xgb",pipeline_xgb_base),
    #("catb",pipeline_catb_base),
    ("lgbm_tune",pipeline_rfe_lgb_tune),
    ("xgb_tune",pipeline_rfe_xgb_tune),
   # ("catb_tune",pipeline_catb_rfe_tune)
]

pipeline_stack = VotingClassifier(estimators=estimators,voting = 'soft',verbose=1,)

cv_score = cross_val_score(pipeline_stack,X_train,y_train,cv = cv,scoring = "accuracy",n_jobs=-1)

In [81]:
cv_score.mean()

0.9967970556287702

In [29]:
pipeline_stack.fit(X_train,y_train)
f1_score(y_test,pipeline_stack.predict(X_test),average = "macro")


[Voting] ..................... (1 of 4) Processing lgbm, total=   3.8s
[Voting] ...................... (2 of 4) Processing xgb, total=   4.8s
[Voting] ................ (3 of 4) Processing lgbm_tune, total=   5.4s
[Voting] ................. (4 of 4) Processing xgb_tune, total=   6.2s


0.9740759781156713

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pipeline_stack.predict(X_test),digits=5))

              precision    recall  f1-score   support

           0    0.99107   0.98813   0.98960       337
           1    0.92256   0.97163   0.94646       564
           2    0.99973   0.99973   0.99973      7305
           3    0.92694   0.80556   0.86200       252
           4    0.99908   0.99967   0.99938     12001
           5    1.00000   0.99809   0.99904       523
           6    0.99820   0.99462   0.99641       558
           7    1.00000   1.00000   1.00000       472

    accuracy                        0.99641     22012
   macro avg    0.97970   0.96968   0.97408     22012
weighted avg    0.99641   0.99641   0.99634     22012



In [31]:
df_test['type_of_attack'] = le.inverse_transform(pipeline_stack.predict(df_test))

In [32]:
df_test.to_csv("../../data/processed/submission.csv",index = False,sep = ";")